In [5]:
import numpy as np
import wandb
import copy
from tqdm import tqdm

In [6]:
api = wandb.Api()

In [7]:
summary_metrics_min = [
    f"n_edges_frac_segment50_{f}"
    for f in [80, 85, 90, 93, 95]
]

In [8]:
def update_summary_metrics(run):
    history = run.history()
    for key in summary_metrics_min:
        try:
            current = copy.copy(history[key])
            if isinstance(current, str):
                continue
            current = current.astype('float').to_numpy()
            if np.isfinite(current).any():
                new = np.nanmean(current).item()
                run.summary[key] = new
        except KeyError:
            pass
        except Exception as e:
            print(key, run)
            raise e
        run.summary.update()

In [12]:
runs = list(api.runs('gnn_tracking/gnn_tracking_gc', {
    "$and": [{
    'created_at': {
        "$gt": '2023-05-31'
        }
    }]
}))
print(f"Found {len(runs)} runs")

Found 73 runs


In [14]:
for run in tqdm(runs):
    update_summary_metrics(run)

100%|██████████| 73/73 [01:46<00:00,  1.45s/it]
